In [2]:
%pip install --quiet -U dotenv
%cd Users/djenk.ivanov/


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
/mnt/batch/tasks/shared/LS_root/mounts/clusters/movielens-compute/code/Users/djenk.ivanov


In [3]:
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Model
from azure.identity import DefaultAzureCredential
import uuid
from azure.ai.ml.entities import Environment
from azure.ai.ml import MLClient
from dotenv import load_dotenv
import os


dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [33]:


custom_env_name = "movielens-recsys-env"

load_dotenv()

credential = DefaultAzureCredential()

custom_job_env = Environment(
    name=custom_env_name,
    description="Environment for MovieLens recommendation system",
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

ml_client = MLClient(
    credential=credential,
    subscription_id=os.getenv("AZURE_SUBSCRIPTION_ID"),
    resource_group_name=os.getenv("AZURE_RESOURCE_GROUP"),
    workspace_name=os.getenv("AZURE_ML_WORKSPACE"),
)

custom_job_env = ml_client.environments.create_or_update(custom_job_env)

print(
    f"Environment with name {custom_job_env.name} is registered to workspace, the environment version is {custom_job_env.version}"
)

Environment with name movielens-recsys-env is registered to workspace, the environment version is 11


# Run job

In [31]:
from azure.ai.ml import command, Input

model_name = "movielens_recommender"
data = Input(type="uri_folder", path="azureml:movielens32m:1")

job = command(
    inputs=dict(
        data=data,
        model_name=model_name,
    ),
    code="./src/",
    command="python main.py --data ${{inputs.data}} --model_name ${{inputs.model_name}}",
    environment="movielens-recsys-env@latest",
    display_name="movielens_recommender_training_job",
)
ml_client.jobs.create_or_update(job)

Experiment,Name,Type,Status,Details Page
djenk,magenta_stick_bgcmsm25h9,command,Starting,Link to Azure Machine Learning studio


# Create endpoint/deploy

In [34]:
endpoint_name = f'movielens-endpoint-{str(uuid.uuid4())[:8]}'

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    auth_mode="key",
)

model_name = "movielens_recommender"

latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=model_name)]
)

ml_client.online_endpoints.begin_create_or_update(endpoint).result()
model = ml_client.models.get(name=model_name, version=latest_model_version)
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    code_path="./src",
    environment="movielens-recsys-env@latest",
    scoring_script="score.py",
    instance_type="Standard_E2s_v3",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(deployment).result()

endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

Check: endpoint movielens-endpoint-da469f0c exists


.....................................................................................................

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://movielens-endpoint-da469f0c.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://movielens-endpoint-da469f0c.eastus.inference.ml.azure.com/swagger.json', 'name': 'movielens-endpoint-da469f0c', 'description': None, 'tags': {}, 'properties': {'createdBy': 'Djenk Ivanov', 'createdAt': '2025-10-31T19:31:36.669132+0000', 'lastModifiedAt': '2025-10-31T19:41:41.539864+0000', 'azureml.onlineendpointid': '/subscriptions/ba117730-305b-41b8-8447-bf6834f0a56f/resourcegroups/djenk-rg1/providers/microsoft.machinelearningservices/workspaces/djenk-ml/onlineendpoints/movielens-endpoint-da469f0c', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ba117730-305b-41b8-8447-bf6834f0a56f/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oeidp:c22f1a9c-ea50-4bf9-b5de-6e7dc424a18d:961f9902-adac-4d91-b699-df54d63afe9c?api-version=2022-0

In [36]:
latest_endpoint_name = "movielens-endpoint-da469f0c"

response = ml_client.online_endpoints.invoke(
    endpoint_name=latest_endpoint_name,
    deployment_name="blue",
    request_file="./payload.json"
)

print(response)

"{\"recommendations\": [{\"movieId\": 50, \"title\": \"Usual Suspects, The (1995)\", \"genres\": \"Crime|Mystery|Thriller\"}, {\"movieId\": 260, \"title\": \"Star Wars: Episode IV - A New Hope (1977)\", \"genres\": \"Action|Adventure|Sci-Fi\"}, {\"movieId\": 296, \"title\": \"Pulp Fiction (1994)\", \"genres\": \"Comedy|Crime|Drama|Thriller\"}, {\"movieId\": 356, \"title\": \"Forrest Gump (1994)\", \"genres\": \"Comedy|Drama|Romance|War\"}, {\"movieId\": 608, \"title\": \"Fargo (1996)\", \"genres\": \"Comedy|Crime|Drama|Thriller\"}, {\"movieId\": 1036, \"title\": \"Die Hard (1988)\", \"genres\": \"Action|Crime|Thriller\"}, {\"movieId\": 1240, \"title\": \"Terminator, The (1984)\", \"genres\": \"Action|Sci-Fi|Thriller\"}, {\"movieId\": 1265, \"title\": \"Groundhog Day (1993)\", \"genres\": \"Comedy|Fantasy|Romance\"}, {\"movieId\": 1270, \"title\": \"Back to the Future (1985)\", \"genres\": \"Adventure|Comedy|Sci-Fi\"}, {\"movieId\": 2762, \"title\": \"Sixth Sense, The (1999)\", \"genres